In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image
import os

class HybridDehazeNet(nn.Module):
    def __init__(self):
        super(HybridDehazeNet, self).__init__()

        # Common layers
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)

        # LightDehaze_Net specific layers
        self.e_conv_layer1 = nn.Conv2d(3, 8, 1, 1, 0, bias=True)
        self.e_conv_layer2 = nn.Conv2d(8, 8, 3, 1, 1, bias=True)
        self.e_conv_layer3 = nn.Conv2d(8, 8, 5, 1, 2, bias=True)
        self.e_conv_layer4 = nn.Conv2d(16, 16, 7, 1, 3, bias=True)
        self.e_conv_layer5 = nn.Conv2d(16, 16, 3, 1, 1, bias=True)
        self.e_conv_layer6 = nn.Conv2d(16, 16, 3, 1, 1, bias=True)
        self.e_conv_layer7 = nn.Conv2d(32, 32, 3, 1, 1, bias=True)
        self.e_conv_layer8 = nn.Conv2d(56, 3, 3, 1, 1, bias=True)

        # Additional layers for the hybrid model
        self.deconv3 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.deconv2 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.deconv1 = nn.Conv2d(64, 3, kernel_size=3, padding=1)

    def forward(self, img):
        # Common feature extraction
        x1 = self.relu(self.conv1(img))
        x2 = self.relu(self.conv2(x1))
        x3 = self.relu(self.conv3(x2))

        # LightDehaze_Net specific feature extraction
        conv_layer1 = self.relu(self.e_conv_layer1(img))
        conv_layer2 = self.relu(self.e_conv_layer2(conv_layer1))
        conv_layer3 = self.relu(self.e_conv_layer3(conv_layer2))
        concat_layer1 = torch.cat((conv_layer1, conv_layer3), 1)
        conv_layer4 = self.relu(self.e_conv_layer4(concat_layer1))
        conv_layer5 = self.relu(self.e_conv_layer5(conv_layer4))
        conv_layer6 = self.relu(self.e_conv_layer6(conv_layer5))
        concat_layer2 = torch.cat((conv_layer4, conv_layer6), 1)
        conv_layer7 = self.relu(self.e_conv_layer7(concat_layer2))
        concat_layer3 = torch.cat((conv_layer2, conv_layer5, conv_layer7), 1)
        conv_layer8 = self.relu(self.e_conv_layer8(concat_layer3))

        # Hybrid model
        hybrid_layer1 = self.relu(self.deconv3(x3))
        hybrid_layer2 = self.relu(self.deconv2(hybrid_layer1))
        hybrid_layer3 = self.relu(self.deconv1(hybrid_layer2))

        dehaze_image = self.relu((conv_layer8 * hybrid_layer3) - conv_layer8 + 1)

        return dehaze_image

# Custom dataset class for input and target pairs
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.hazy_folder = os.path.join(root_dir, 'hazy')
        self.clear_folder = os.path.join(root_dir, 'clear')
        self.hazy_images = os.listdir(self.hazy_folder)

    def __len__(self):
        return len(self.hazy_images)

    def __getitem__(self, idx):
        hazy_img_name = self.hazy_images[idx]
        hazy_path = os.path.join(self.hazy_folder, hazy_img_name)
        clear_path = os.path.join(self.clear_folder, hazy_img_name)  # Assuming filenames match

        hazy_image = Image.open(hazy_path).convert("RGB")
        clear_image = Image.open(clear_path).convert("RGB")

        if self.transform:
            hazy_image = self.transform(hazy_image)
            clear_image = self.transform(clear_image)

        return hazy_image, clear_image

# Set device (CPU/GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define data transformations
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Create custom dataset
train_dataset = CustomDataset('/content/drive/MyDrive/Data/Train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)

# Initialize the model, loss function, and optimizer
model = HybridDehazeNet().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    for batch_idx, (hazy, clear) in enumerate(train_loader):
        hazy, clear = hazy.to(device), clear.to(device)

        # Forward pass
        output = model(hazy)

        # Compute the loss
        loss = criterion(output, clear)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print training information
        if batch_idx % 100 == 0:
            print(f'Epoch [{epoch}/{num_epochs}], Batch [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Save the trained model
torch.save(model.state_dict(), 'hybrid_dehazenet_model.pth')


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Data/dehazenet_model.pth'

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import os

# Load the trained model
model = LightweightDehazeNet()
model.load_state_dict(torch.load('lightweight_dehazenet_model.pth'))
model.eval()

# Set device (CPU/GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define transformation for inference
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5, 0.5])
])

# Denormalization function to convert tensor back to image
def denormalize(tensor):
    tensor = tensor.cpu().numpy().transpose(1, 2, 0)
    tensor = (tensor + 1) / 2.0  # Denormalize to [0, 1] range
    tensor = (tensor * 255).astype('uint8')
    return Image.fromarray(tensor)

# Function to dehaze a single image
def dehaze_image(model, image_path, transform, device):
    hazy_image = Image.open(image_path).convert("RGBA")  # Assuming 4-channel images
    hazy_image = transform(hazy_image).unsqueeze(0).to(device)

    with torch.no_grad():
        model.eval()
        dehazed_image = model(hazy_image)

    dehazed_image = denormalize(dehazed_image.squeeze(0))

    return dehazed_image

# Directory containing hazy images for dehazing
hazy_images_dir = '/content/drive/MyDrive/Data/Train/hazy'

# Directory to save dehazed images
dehazed_images_dir = '/content/results4'

# Create the output directory if it doesn't exist
os.makedirs(dehazed_images_dir, exist_ok=True)

# Dehaze each image in the directory
for hazy_image_name in os.listdir(hazy_images_dir):
    hazy_image_path = os.path.join(hazy_images_dir, hazy_image_name)
    dehazed_image = dehaze_image(model, hazy_image_path, transform, device)

    # Save the dehazed image
    dehazed_image_name = f'dehazed_{hazy_image_name}'
    dehazed_image_path = os.path.join(dehazed_images_dir, dehazed_image_name)
    dehazed_image.save(dehazed_image_path)

    print(f'Dehazed image saved: {dehazed_image_path}')


Dehazed image saved: /content/results4/dehazed_18.png
Dehazed image saved: /content/results4/dehazed_13.png
Dehazed image saved: /content/results4/dehazed_12.png
Dehazed image saved: /content/results4/dehazed_11.png
Dehazed image saved: /content/results4/dehazed_17.png
Dehazed image saved: /content/results4/dehazed_10.png
Dehazed image saved: /content/results4/dehazed_1.png
Dehazed image saved: /content/results4/dehazed_19.png
Dehazed image saved: /content/results4/dehazed_16.png
Dehazed image saved: /content/results4/dehazed_14.png
Dehazed image saved: /content/results4/dehazed_15.png
Dehazed image saved: /content/results4/dehazed_21.png
Dehazed image saved: /content/results4/dehazed_20.png
Dehazed image saved: /content/results4/dehazed_2.png
Dehazed image saved: /content/results4/dehazed_24.png
Dehazed image saved: /content/results4/dehazed_23.png
Dehazed image saved: /content/results4/dehazed_22.png
Dehazed image saved: /content/results4/dehazed_28.png
Dehazed image saved: /content/

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
import os

# Define the dehazing model architecture
class DehazeNet(nn.Module):
    def __init__(self):
        super(DehazeNet, self).__init__()

        # Encoder layers
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)

        # Decoder layers
        self.deconv3 = nn.ConvTranspose2d(256, 128, kernel_size=3, stride=1, padding=1)
        self.deconv2 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=1, padding=1)
        self.deconv1 = nn.ConvTranspose2d(64, 3, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        # Encoder
        x1 = F.relu(self.conv1(x))
        x2 = F.relu(self.conv2(x1))
        x3 = F.relu(self.conv3(x2))

        # Decoder
        x_deconv3 = F.relu(self.deconv3(x3))
        x_deconv2 = F.relu(self.deconv2(x_deconv3 + x2))
        x_deconv1 = F.sigmoid(self.deconv1(x_deconv2 + x1))

        return x_deconv1

# Load the trained model
model = DehazeNet()
model.load_state_dict(torch.load('dehazenet_model.pth'))
model.eval()

# Set device (CPU/GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define transformation for inference
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Denormalization function to convert tensor back to image
def denormalize(tensor):
    tensor = tensor.cpu().numpy().transpose(1, 2, 0)
    tensor = (tensor + 1) / 2.0  # Denormalize to [0, 1] range
    tensor = (tensor * 255).astype('uint8')
    return Image.fromarray(tensor)

# Function to dehaze a single image
def dehaze_image(model, image_path, transform, device):
    hazy_image = Image.open(image_path).convert("RGB")  # Assuming 3-channel images
    hazy_image = transform(hazy_image).unsqueeze(0).to(device)

    with torch.no_grad():
        model.eval()
        dehazed_image = model(hazy_image)

    dehazed_image = denormalize(dehazed_image.squeeze(0))

    return dehazed_image

# Directory containing hazy images for dehazing
hazy_images_dir = '/content/drive/MyDrive/Data/Test/hazy'

# Directory to save dehazed images
dehazed_images_dir = '/content/results4'

# Create the output directory if it doesn't exist
os.makedirs(dehazed_images_dir, exist_ok=True)

# Dehaze each image in the directory
for hazy_image_name in os.listdir(hazy_images_dir):
    hazy_image_path = os.path.join(hazy_images_dir, hazy_image_name)
    dehazed_image = dehaze_image(model, hazy_image_path, transform, device)

    # Save the dehazed image
    dehazed_image_name = f'dehazed_{hazy_image_name}'
    dehazed_image_path = os.path.join(dehazed_images_dir, dehazed_image_name)
    dehazed_image.save(dehazed_image_path)

    print(f'Dehazed image saved: {dehazed_image_path}')
